# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,10.09,71,0,2.57,CA,1719747046
1,1,puerto natales,-51.7236,-72.4875,4.00,83,99,5.27,CL,1719747048
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.13,97,100,8.13,SH,1719747050
3,3,tiksi,71.6872,128.8694,8.58,54,34,1.66,RU,1719747052
4,4,colby,39.3958,-101.0524,15.56,80,0,5.14,US,1719747054


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display
#city_data_df = city_data_df.dropna(subset=['Lat', 'Lng', 'Humidity'])

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', 
                           geo=True,
                           tiles='OSM', 
                           frame_width=700,
                           frame_height=500, 
                           size='Humidity',
                           scale = 0.50,
                           color='City',
                           alpha=0.5
                          )
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#Selected Criteria:
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
Listed_cities = city_data_df[
                                (city_data_df['Max Temp'] >= 21) & 
                                (city_data_df['Max Temp'] <= 27) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)
                            ]
# Drop any rows with null values
Listed_cities = Listed_cities.dropna()

# Display sample data
Listed_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,kailua-kona,19.6406,-155.9956,23.98,78,0,2.06,US,1719747080
98,98,mahina,-17.5065,-149.4890,25.87,61,0,3.09,PF,1719747028
105,105,lake charles,30.2131,-93.2044,26.99,96,0,1.54,US,1719747018
106,106,hamilton,39.1834,-84.5333,23.97,95,0,0.89,US,1719747263
168,168,ribeira brava,32.6500,-17.0667,23.89,79,0,4.06,PT,1719747394
300,300,holualoa,19.6228,-155.9522,21.91,93,0,2.06,US,1719747672
313,313,caconda,-13.7333,15.0667,25.88,17,0,2.25,AO,1719747697
315,315,karratha,-20.7377,116.8463,21.93,38,0,3.00,AU,1719747701
410,410,laguna,38.4210,-121.4238,22.02,68,0,0.00,US,1719747895
458,458,orapa,-21.2833,25.3667,25.14,20,0,1.37,BW,1719748042


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Listed_cities[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,kailua-kona,US,19.6406,-155.9956,78,
98,mahina,PF,-17.5065,-149.4890,61,
105,lake charles,US,30.2131,-93.2044,96,
106,hamilton,US,39.1834,-84.5333,95,
168,ribeira brava,PT,32.6500,-17.0667,79,
300,holualoa,US,19.6228,-155.9522,93,
313,caconda,AO,-13.7333,15.0667,17,
315,karratha,AU,-20.7377,116.8463,38,
410,laguna,US,38.4210,-121.4238,68,
458,orapa,BW,-21.2833,25.3667,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

#Params for type of place
categories = "accommodation.hotel"

#Params for type of search
radius = 10000

params = {
            "apiKey": geoapify_key,
            "categories":categories,
          }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params =params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
mahina - nearest hotel: Motu Martin
lake charles - nearest hotel: Howard Johnson by Wyndham Historic Lake Charles
hamilton - nearest hotel: North Vista Manor
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
holualoa - nearest hotel: Kona Hotel
caconda - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
orapa - nearest hotel: No hotel found
itaituba - nearest hotel: TG Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
17,kailua-kona,US,19.6406,-155.9956,78,Kona Seaside Hotel
98,mahina,PF,-17.5065,-149.4890,61,Motu Martin
105,lake charles,US,30.2131,-93.2044,96,Howard Johnson by Wyndham Historic Lake Charles
106,hamilton,US,39.1834,-84.5333,95,North Vista Manor
168,ribeira brava,PT,32.6500,-17.0667,79,Cheerfulway Bravamar Hotel
300,holualoa,US,19.6228,-155.9522,93,Kona Hotel
313,caconda,AO,-13.7333,15.0667,17,No hotel found
315,karratha,AU,-20.7377,116.8463,38,Karratha International Hotel
410,laguna,US,38.4210,-121.4238,68,Holiday Inn Express & Suites
458,orapa,BW,-21.2833,25.3667,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
                                    'Lng', 'Lat', 
                                    geo=True,
                                    tiles='OSM', 
                                    frame_width=700,
                                    frame_height=500, 
                                    size='Humidity',
                                    scale = 1.0,
                                    alpha=0.6,
                                    color='City',
                                    hover_cols = ["Hotel Name", "Country"]
                                  )


# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)